In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import pandas as pd
import dbutils
import csv
from sklearn.utils import class_weight
import seaborn as sns
from pylab import rcParams
import warnings
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, roc_curve


In [ ]:
%matplotlib inline
rcParams['figure.figsize'] = 10, 6
warnings.filterwarnings('ignore')
sns.set(style="darkgrid")


In [ ]:
db_connection_file = 'prod_dbconnection.json'
#connector = dbutils.connect_to_main_database(db_connection_file)
#select_personlab="SELECT m_truth.age,abs(m_truth.height - ar_height.float_value) AS ae, ar_pose.json_value->'pose_1' -> 'nose'->'score' as nose,ar_pose.json_value->'pose_1' -> 'leftAnkle'->'score',ar_pose.json_value->'pose_1' -> 'leftEar'->'score',ar_pose.json_value->'pose_1' -> 'leftElbow'->'score',ar_pose.json_value->'pose_1' -> 'leftEye'->'score',ar_pose.json_value->'pose_1' -> 'leftHip'->'score',ar_pose.json_value->'pose_1' -> 'leftKnee'->'score',ar_pose.json_value->'pose_1' -> 'leftShoulder'->'score',ar_pose.json_value->'pose_1' -> 'leftWrist'->'score',ar_pose.json_value->'pose_1' -> 'rightAnkle'->'score',ar_pose.json_value->'pose_1' -> 'rightEar'->'score',ar_pose.json_value->'pose_1' -> 'rightElbow'->'score',ar_pose.json_value->'pose_1' -> 'rightEye'->'score',ar_pose.json_value->'pose_1' -> 'rightHip'->'score',ar_pose.json_value->'pose_1' -> 'rightKnee'->'score',ar_pose.json_value->'pose_1' -> 'rightShoulder'->'score',ar_pose.json_value->'pose_1' -> 'rightWrist'->'score' FROM measure m_scan JOIN measure m_truth ON m_scan.qr_code::text = m_truth.qr_code::text AND m_scan.age = m_truth.age JOIN artifact a_pcd ON a_pcd.measure_id = m_scan.id::text JOIN artifact_result ar_height ON ar_height.artifact_id::text = a_pcd.id::text JOIN artifact a_rgb ON a_pcd.measure_id = a_rgb.measure_id AND round(a_pcd.session_timestamp::numeric, 2) = round(a_rgb.session_timestamp::numeric, 2) JOIN artifact_result ar_pose ON ar_pose.artifact_id::text = a_rgb.id::text WHERE a_pcd.dataformat = 'pcd'::text AND a_rgb.dataformat = 'rgb'::text AND m_truth.type = 'manual'::text AND ar_pose.model_id ~~ '%pose%'::text AND ar_height.model_id like '%GAP%' AND ar_pose.float_value=1";
#data = pd.DataFrame(connector.execute(select_personlab, fetch_all=True), columns=['age','error','json_value_pose_1_nose_score','json_value_pose_1_leftAnkle_score','json_value_pose_1_leftEar_score','json_value_pose_1_leftElbow_score','json_value_pose_1_leftEye_score','json_value_pose_1_leftHip_score','json_value_pose_1_leftKnee_score','json_value_pose_1_leftShoulder_score','json_value_pose_1_leftWrist_score','json_value_pose_1_rightAnkle_score','json_value_pose_1_rightEar_score','json_value_pose_1_rightElbow_score','json_value_pose_1_rightEye_score','json_value_pose_1_rightHip_score','json_value_pose_1_rightKnee_score','json_value_pose_1_rightShoulder_score','json_value_pose_1_rightWrist_score'])



In [ ]:
#data.to_csv('posentvsgapnet_2.csv')

In [ ]:
#please prepare data or ask prajwal for data

In [ ]:
data = pd.read_csv('posentvsgapnet_2.csv')
#852

In [ ]:
data.shape

In [ ]:
#data['scan_step_x']=data['scan_step_x']%100

In [ ]:
data.head()

In [ ]:
data['gold']=np.where(data['error']<1.2,1,0)

In [ ]:
mean=data.mean(axis=0)
std=data.std(axis=0)

In [ ]:
data['age']-=mean['age']
data['age']/=std['age']

In [ ]:
data.head()

In [ ]:
del data['error']

In [ ]:
X = data.loc[:, data.columns!='gold']

In [ ]:
Y = data.loc[:, data.columns=='gold']

In [ ]:
X.head()

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.2,random_state=42)


In [ ]:

ax = sns.countplot(x='gold', data=data)
print(Y_test['gold'].value_counts())


In [ ]:
data['gold'].value_counts()[0]/data.shape[0]

In [ ]:
data['gold'].value_counts()[1]/data.shape[0]

In [ ]:
clf = RandomForestClassifier(n_estimators=10000, 
                               random_state=50, 
                               max_features = 'sqrt',
                               n_jobs=4, verbose = 1)

In [ ]:
clf.fit(X_train,Y_train)

In [ ]:
#import pickle
#filename = 'finalized_model.sav'
#pickle.dump(clf, open(filename, 'wb'))


In [ ]:
Y_test_pred = clf.predict(X_test).tolist()

In [ ]:
val=[i for i in Y_test.gold]

In [ ]:
pd.crosstab( pd.Series(val, name = 'Actual'),
    pd.Series(Y_test_pred, name = 'Predicted')
           )

In [ ]:
names=['age','json_value_pose_1_leftAnkle_score','json_value_pose_1_leftEye_score','json_value_pose_1_leftHip_score','json_value_pose_1_leftKnee_score','json_value_pose_1_leftShoulder_score','json_value_pose_1_rightAnkle_score','json_value_pose_1_rightEye_score','json_value_pose_1_rightHip_score','json_value_pose_1_rightKnee_score','json_value_pose_1_rightShoulder_score']


In [ ]:
print (sorted(zip(map(lambda x: round(x, 4), clf.feature_importances_), names), 
             reverse=True))

In [ ]:
def generate_model_report(y_actual, y_predicted):
    print("Accuracy = " , accuracy_score(y_actual, y_predicted))
    print("Precision = " ,precision_score(y_actual, y_predicted))
    print("Recall = " ,recall_score(y_actual, y_predicted))
    print("F1 Score = " ,f1_score(y_actual, y_predicted))
    pass

In [ ]:
def generate_auc_roc_curve(clf, X_test):
    y_pred_proba = clf.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(Y_test,  y_pred_proba)
    auc = roc_auc_score(Y_test, y_pred_proba)
    plt.plot(fpr,tpr,label="AUC ROC Curve with Area Under the curve ="+str(auc))
    plt.legend(loc=4)
    plt.show()
    pass

In [ ]:
generate_model_report(Y_test, Y_test_pred)

In [ ]:
generate_auc_roc_curve(clf, X_test)

# class_weight='balanced'

In [ ]:
unique_classes = list(data['gold'].unique())
unique_classes

In [ ]:
out_dict = {}
for classes in unique_classes:
    out_dict[classes] = data.shape[0]/((data.loc[data['gold'] == classes].shape[0])
                                     *len(unique_classes))

In [ ]:
out_dict

In [ ]:
print(X_train.shape,Y_train.shape)

In [ ]:
clf = RandomForestClassifier(n_estimators=10000, 
                               random_state=50, 
                               max_features = 'sqrt',
                               n_jobs=4, verbose = 1,
                            class_weight='balanced').fit(X_train,Y_train)

In [ ]:
from sklearn.utils import class_weight


In [ ]:
class_weight.compute_class_weight('balanced', np.unique(Y_train), Y_train['gold'])

In [ ]:
Y_test_pred=clf.predict(X_test)

In [ ]:
val=[i for i in Y_test.gold]
pd.crosstab( pd.Series(val, name = 'Actual'),
    pd.Series(Y_test_pred, name = 'Predicted')
           )

In [ ]:
generate_model_report(Y_test, Y_test_pred)

In [ ]:
generate_auc_roc_curve(clf, X_test)

In [ ]:
from sklearn.model_selection import GridSearchCV


In [ ]:
'''weights = np.linspace(0.05, 0.95, 20)
gsc = GridSearchCV(
    estimator=RandomForestClassifier(n_estimators=10000, 
                               random_state=50, 
                               max_features = 'sqrt',
                               n_jobs=4, verbose = 1),
    param_grid={
        'class_weight': [{0: x, 1: 1.0-x} for x in weights]
    },
    scoring='f1',
    cv=5
)

grid_result = gsc.fit(X_train, Y_train)
print("Best parameters : %s" % grid_result.best_params_)
'''

In [ ]:
data_out=pd.DataFrame({'score':grid_result.cv_results_['mean_test_score'],
                      'weight':weights})
data_out.plot(x='weight')

In [ ]:
clf = RandomForestClassifier(n_estimators=10000, 
                               random_state=50, 
                               max_features = 'sqrt',
                               n_jobs=4, verbose = 1,
                            **grid_result.best_params_).fit(X_train,Y_train)

In [ ]:
val=[i for i in Y_test.gold]
pd.crosstab( pd.Series(val, name = 'Actual'),
    pd.Series(Y_test_pred, name = 'Predicted')
           )

In [ ]:
generate_model_report(Y_test, Y_test_pred)

In [ ]:
generate_auc_roc_curve(clf, X_test)

# SMOTE

In [ ]:
unique, count = np.unique(Y_train, return_counts=True)
Y_train_dict_value_count = { k:v for (k,v) in zip(unique, count)}
Y_train_dict_value_count

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
sm = SMOTE(random_state=12)
X,Y = sm.fit_sample(X, Y)

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.2,random_state=42)

In [ ]:
unique, count = np.unique(Y, return_counts=True)
y_train_smote_value_count = { k:v for (k,v) in zip(unique, count)}
y_train_smote_value_count

In [ ]:
clf = RandomForestClassifier(n_estimators=50000, 
                               random_state=50, 
                               max_features = 'sqrt',
                               n_jobs=4, verbose = 1,
                            class_weight='balanced').fit(X_train,Y_train)

In [ ]:
Y_test_pred=clf.predict(X_test)

In [ ]:
val=[i for i in Y_test.gold]
pd.crosstab( pd.Series(val, name = 'Actual'),
    pd.Series(Y_test_pred, name = 'Predicted')
           )

In [ ]:
#Y_test_pred=clf.predict(X_train)
#val=[i for i in Y_train.gold]
#pd.crosstab( pd.Series(val, name = 'Actual'),
#    pd.Series(Y_test_pred, name = 'Predicted')
#           )

In [ ]:
generate_model_report(Y_test, Y_test_pred)

In [ ]:
generate_auc_roc_curve(clf, X_test)

In [ ]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
h